In [20]:
import numpy as np
import skimage as sk
from glob import glob
import napari
from cellpose import io, models, train
import os


In [21]:
img_files = sorted(glob("../Microglia_CellPose_Model/Images/*.tif"))
mask_files = sorted(glob("../Microglia_CellPose_Model/Masks/*.tif"))
imgs = list(map(sk.io.imread, img_files))
masks = list(map(sk.io.imread, mask_files))

In [23]:
os.mkdir('../Microglia_CellPose_Model/Training')
save = '../Microglia_CellPose_Model/Training'
unstacked_imgs = []
unstacked_masks = []
for i in range(len(imgs)):
    split_img = np.split(imgs[i], imgs[i].shape[0],axis=0)
    split_mask = np.split(masks[i], masks[i].shape[0],axis=0)
    for c in range(len(split_img)):
        sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+".tif"),split_img[c])
        sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+"_mask.tif"),split_img[c])

41